# Study of the effect of national factors on home prices in the US

### Task: 
Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

### Approach:
The following variables are chosen for the study:

1.  Unemployment Rate
2.  Employment Rate
3.  Per capita GDP
4.  Median Household Income
5.  Construction Prices
6.  CPI
7.  Interest Rates
8.  The number of new houses supplied
9.  Working Population
10. Urban Population
11. Percentage of population above 65
12. Housing subsidies
13. Number of Households

As a proxy for home prices, the S&P Case-Shiller Index is used.

Note: Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the Extract Transform Load (ETL) method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.

## Importing neccessary libraries

In [1]:
!pip install pandas_datareader --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as pdr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
import datetime

In [3]:
start = datetime.datetime(2003, 1, 1)
end = datetime.datetime(2023, 12, 31)

## Perform ETL

In [4]:
import pandas as pd
import pandas_datareader.data as web
# Reading CASE-SHILLER Index into a dataframe
fred_code = "CSUSHPISA" 
start_date = "2003-01-01"
end_date = "2023-12-31"

df_CS = web.DataReader(fred_code, "fred", start_date, end_date)

df_CS.reset_index(inplace=True)

df_CS.rename(columns={fred_code: "CS_Index_Value"}, inplace=True)

df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month

print("Shape of the CASE-SHILLER Index DataFrame:", df_CS.shape)
print("\nLast 5 rows of the DataFrame:")
df_CS.tail()

Shape of the CASE-SHILLER Index DataFrame: (252, 4)

Last 5 rows of the DataFrame:


,DATE,CS_Index_Value,Year,Month
247,2023-08-01,309.045,2023,8
248,2023-09-01,311.176,2023,9
249,2023-10-01,313.262,2023,10
250,2023-11-01,314.268,2023,11
251,2023-12-01,315.075,2023,12


In [5]:
fred_code_unemp = "UNRATE" # Civilian Unemployment Rate

df_unemp = web.DataReader(fred_code_unemp, "fred", start_date, end_date)

df_unemp.reset_index(inplace=True)

df_unemp.rename(columns={fred_code_unemp: "Unemployment_Rate"}, inplace=True)

print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (252, 2)


,DATE,Unemployment_Rate
247,2023-08-01,3.7
248,2023-09-01,3.8
249,2023-10-01,3.9
250,2023-11-01,3.7
251,2023-12-01,3.8


In [6]:
fred_code_emp = "EMRATIO" # Reading Employment Rate Data into a dataframe

df_emp = web.DataReader(fred_code_emp, "fred", start_date, end_date)

df_emp.reset_index(inplace=True)

df_emp = df_emp.rename(columns={fred_code_emp: 'EmpRate'})

print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (252, 2)


,DATE,EmpRate
247,2023-08-01,60.4
248,2023-09-01,60.4
249,2023-10-01,60.2
250,2023-11-01,60.4
251,2023-12-01,60.1


In [7]:
fred_code_pcgdp = "A939RX0Q048SBEA" # Real Gross Domestic Product per Capita

df_pcgdp = web.DataReader(fred_code_pcgdp, "fred", start_date, end_date)

df_pcgdp.reset_index(inplace=True)

df_pcgdp = df_pcgdp.rename(columns={fred_code_pcgdp: 'Per_Capita_GDP'})

print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (84, 2)


,DATE,Per_Capita_GDP
79,2022-10-01,66341
80,2023-01-01,66673
81,2023-04-01,66945
82,2023-07-01,67499
83,2023-10-01,67858


In [8]:
fred_code_fed_rate = "FEDFUNDS" # Interest Rate Data

df_Fed_rate = web.DataReader(fred_code_fed_rate, "fred", start_date, end_date)

df_Fed_rate.reset_index(inplace=True)

df_Fed_rate.rename(columns={fred_code_fed_rate: "FEDFUNDS"}, inplace=True)

print("Shape of the Interest rate data:- ", df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (252, 2)


,DATE,FEDFUNDS
247,2023-08-01,5.33
248,2023-09-01,5.33
249,2023-10-01,5.33
250,2023-11-01,5.33
251,2023-12-01,5.33


In [9]:
fred_code_cons_price = "WPUSI012011" # Producer Price Index by Commodity: Construction Materials

df_cons_price_index = web.DataReader(fred_code_cons_price, "fred", start_date, end_date)

df_cons_price_index.reset_index(inplace=True)

df_cons_price_index = df_cons_price_index.rename(columns={fred_code_cons_price: 'Cons_Material'})

print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (252, 2)


,DATE,Cons_Material
247,2023-08-01,333.794
248,2023-09-01,332.098
249,2023-10-01,328.743
250,2023-11-01,326.844
251,2023-12-01,327.644


In [10]:
fred_code_cpi = "CPIAUCSL" # Consumer Price Index

df_CPI = web.DataReader(fred_code_cpi, "fred", start_date, end_date).reset_index()

df_CPI = df_CPI.rename(columns={fred_code_cpi: 'CPI'})

print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (252, 2)


,DATE,CPI
247,2023-08-01,306.138
248,2023-09-01,307.374
249,2023-10-01,307.653
250,2023-11-01,308.087
251,2023-12-01,308.735


In [11]:
fred_code_house = "MSACSR" # Monthly new house supply

df_house = web.DataReader(fred_code_house, "fred", start_date, end_date).reset_index()

df_house = df_house.rename(columns={fred_code_house: 'Houses'})

print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()

Shape of the monthly house supply data:-  (252, 2)


,DATE,Houses
247,2023-08-01,8.0
248,2023-09-01,7.7
249,2023-10-01,7.8
250,2023-11-01,8.8
251,2023-12-01,8.4


In [12]:
# Population above 65
df_oldpop = pd.read_csv("old_age_pop.csv", names=["DATE", "old_age_pop"], skiprows=1) 

df_oldpop['DATE'] = pd.to_datetime(df_oldpop['DATE'].astype(str) + '-01-01', format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

df_oldpop['old_age_pop'] = df_oldpop['old_age_pop'].round(1)

print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

Shape of the population data age above 65:-  (21, 2)


,DATE,old_age_pop
16,2019-01-01,15.7
17,2020-01-01,16.1
18,2021-01-01,16.5
19,2022-01-01,16.9
20,2023-01-01,17.4


In [13]:
df_urban = pd.read_csv("urban_pop.csv", names=["DATE", "urban_pop_us"], skiprows=1) # Urban Population Percent

df_urban['DATE'] = pd.to_datetime(df_urban['DATE'].astype(str) + '-01-01', format="%Y-%m-%d").dt.strftime("%Y-%m-%d")

df_urban['urban_pop_us'] = df_urban['urban_pop_us'].round(1)

print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

Shape of the urban population percent data:-  (21, 2)


,DATE,urban_pop_us
16,2019-01-01,82.5
17,2020-01-01,82.7
18,2021-01-01,82.9
19,2022-01-01,83.1
20,2023-01-01,83.3


In [14]:
df_subsidy = pd.read_csv("housing_subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1) # Housing Subsidies
df_subsidy['DATE'] = pd.to_datetime(df_urban['DATE'], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()

Shape of the housing subsidies:-  (21, 2)


,DATE,Subsidy
16,2019-01-01,40.185
17,2020-01-01,44.147
18,2021-01-01,45.299
19,2022-01-01,48.021
20,2023-01-01,53.573


In [15]:
df_working = pd.read_csv("working_age_population.csv", names = ["DATE", "working_age_pop"], skiprows = 1) # Working age population
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (241, 2)


,DATE,working_age_pop
236,01/09/2022,207518300
237,01/10/2022,207577400
238,01/11/2022,207627300
239,01/12/2022,207658500
240,01/01/2023,208262400


In [16]:
# Real Median Household Income
df_income = pd.read_csv("median_household_income.csv", names = ["DATE", "median_income"], skiprows = 1) 
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()

Shape of the median household income data:-  (21, 2)


,DATE,median_income
16,01/01/2019,81210
17,01/01/2020,79560
18,01/01/2021,79260
19,01/01/2022,77540
20,01/01/2023,80610


In [17]:
# Total number of households
df_households = pd.read_csv("household.csv", names = ["DATE", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()

Shape of the total households data:-  (21, 2)


,DATE,Num_Households
16,01/01/2019,128579
17,01/01/2020,128451
18,01/01/2021,129224
19,01/01/2022,131202
20,01/01/2023,131434


In [18]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()

,DATE,CS_Index_Value,Year,Month,Per_Capita_GDP
0,2003-01-01,128.461,2003,1,50462.0
1,2003-02-01,129.356,2003,2,NaN
2,2003-03-01,130.148,2003,3,NaN
3,2003-04-01,130.884,2003,4,50796.0
4,2003-05-01,131.735,2003,5,NaN


In [19]:
import pandas as pd
import re
# function to correct the date format 
def standardize_first_day_dates(date_str):
    """
    Standardize dates to YYYY-MM-DD format
    Handles dates that are either:
    - First day of month: YYYY-MM-01 
    - First day of year: YYYY-01-01
    - With separators: '-' or '\'
    - In formats: YYYY-MM-DD, YYYY-DD-MM, MM-DD-YYYY, DD-MM-YYYY, etc.
    """
    if pd.isna(date_str) or date_str == '':
        return date_str
    
    # Convert to string and clean
    date_str = str(date_str).strip()
    
    # Replace backslashes with dashes for consistent processing
    date_str = date_str.replace('\\', '-').replace('/', '-')
    
    # Split the date parts
    parts = date_str.split('-')
    
    if len(parts) != 3:
        return date_str  # Return original if not 3 parts
    
    # Convert parts to integers for processing
    try:
        part1, part2, part3 = int(parts[0]), int(parts[1]), int(parts[2])
    except ValueError:
        return date_str  # Return original if conversion fails
    
    # Identify which part is the year (4 digits or > 31)
    year, month, day = None, None, None
    
    # Find the year (typically 4 digits or > 31)
    if part1 > 31 or len(parts[0]) == 4:
        year = part1
        # For YYYY-MM-DD or YYYY-DD-MM format
        if part2 == 1 and part3 == 1:
            # YYYY-01-01 (first day of year)
            month, day = 1, 1
        elif part3 == 1:
            # YYYY-MM-01 (first day of month)
            month, day = part2, 1
        elif part2 == 1:
            # YYYY-01-DD format, but since we expect first days, treat as YYYY-01-MM
            month, day = part3, 1
        else:
            # Default: assume YYYY-MM-DD
            month, day = part2, part3
            
    elif part2 > 31 or len(parts[1]) == 4:
        year = part2
        # For MM-YYYY-DD or DD-YYYY-MM format (less common)
        if part1 == 1 and part3 == 1:
            month, day = 1, 1
        elif part3 == 1:
            month, day = part1, 1
        elif part1 == 1:
            month, day = part3, 1
        else:
            month, day = part1, part3
            
    elif part3 > 31 or len(parts[2]) == 4:
        year = part3
        # For MM-DD-YYYY or DD-MM-YYYY format
        if part1 == 1 and part2 == 1:
            month, day = 1, 1
        elif part2 == 1:
            month, day = part1, 1
        elif part1 == 1:
            month, day = part2, 1
        else:
            # Need to determine if it's MM-DD-YYYY or DD-MM-YYYY
            if part1 > 12:  # First part is day
                month, day = part2, part1
            else:  # First part is month
                month, day = part1, part2
    else:
        # No clear year indicator, assume first part is year if >= 1900
        if part1 >= 1900:
            year = part1
            if part2 == 1 and part3 == 1:
                month, day = 1, 1
            elif part3 == 1:
                month, day = part2, 1
            elif part2 == 1:
                month, day = part3, 1
            else:
                month, day = part2, part3
        else:
            return date_str  # Can't determine format
    
    # Ensure month is between 1-12
    if month > 12:
        month, day = day, month  # Swap if month > 12
    
    # Format as YYYY-MM-DD
    try:
        return f"{year:04d}-{month:02d}-{day:02d}"
    except:
        return date_str  # Return original if formatting fails

In [20]:
df_working['DATE'] = df_working['DATE'].apply(standardize_first_day_dates)

In [21]:
df_working

,DATE,working_age_pop
0,2003-01-01,185656300
1,2003-02-01,185830200
2,2003-03-01,186120800
3,2003-04-01,186475100
4,2003-05-01,186696400
...,...,...
236,2022-09-01,207518300
237,2022-10-01,207577400
238,2022-11-01,207627300
239,2022-12-01,207658500


In [22]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(252, 11)


,CS_Index_Value,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Unemployment_Rate,EmpRate,Cons_Material,FEDFUNDS
DATE,,,,,,,,,,,
2003-01-01,128.461,2003,1,50462.0,185656300.0,4.0,182.6,5.8,62.5,144.4,1.24
2003-02-01,129.356,2003,2,NaN,185830200.0,4.5,183.6,5.9,62.5,145.2,1.26
2003-03-01,130.148,2003,3,NaN,186120800.0,4.1,183.9,5.9,62.4,145.2,1.25
2003-04-01,130.884,2003,4,50796.0,186475100.0,4.1,183.2,6.0,62.4,145.9,1.26
2003-05-01,131.735,2003,5,NaN,186696400.0,3.9,182.9,6.1,62.3,145.8,1.26


In [23]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CS_Index_Value,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Unemployment_Rate,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2003-01-01,128.461,2003,1,50462.0,185656300.0,4.0,182.6,5.8,62.5,144.4,1.24,79.6,111278,68350,25.93,12.1
2003-02-01,129.356,2003,2,NaN,185830200.0,4.5,183.6,5.9,62.5,145.2,1.26,79.6,111278,68350,25.93,12.1
2003-03-01,130.148,2003,3,NaN,186120800.0,4.1,183.9,5.9,62.4,145.2,1.25,79.6,111278,68350,25.93,12.1
2003-04-01,130.884,2003,4,50796.0,186475100.0,4.1,183.2,6.0,62.4,145.9,1.26,79.6,111278,68350,25.93,12.1
2003-05-01,131.735,2003,5,NaN,186696400.0,3.9,182.9,6.1,62.3,145.8,1.26,79.6,111278,68350,25.93,12.1


In [24]:
df.shape

(252, 16)

Checking missing values (NAN)

In [25]:
df.isnull().sum()

CS_Index_Value         0
Year                   0
Month                  0
Per_Capita_GDP       168
working_age_pop       11
Houses                 0
CPI                    0
Unemployment_Rate      0
EmpRate                0
Cons_Material          0
FEDFUNDS               0
urban_pop_us           0
Num_Households         0
median_income          0
Subsidy                0
old_age_pop            0
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other column is due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation and then same thing will be applied on "working_age_pop".  This means that we will use data from 2003 to 2023.

### Interpolation:

Interpolation is a mathematical technique used to estimate values that are missing in a dataset based on the values of neighboring data points. It calculates intermediate values based on the existing data.

In [26]:
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [28]:
df["working_age_pop"] = df["working_age_pop"].interpolate()

In [29]:
df

,CS_Index_Value,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Unemployment_Rate,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2003-01-01,128.461,2003,1,50462.000000,185656300.0,4.0,182.600,5.8,62.5,144.400,1.24,79.6,111278,68350,25.930,12.1
2003-02-01,129.356,2003,2,50573.333333,185830200.0,4.5,183.600,5.9,62.5,145.200,1.26,79.6,111278,68350,25.930,12.1
2003-03-01,130.148,2003,3,50684.666667,186120800.0,4.1,183.900,5.9,62.4,145.200,1.25,79.6,111278,68350,25.930,12.1
2003-04-01,130.884,2003,4,50796.000000,186475100.0,4.1,183.200,6.0,62.4,145.900,1.26,79.6,111278,68350,25.930,12.1
2003-05-01,131.735,2003,5,51034.666667,186696400.0,3.9,182.900,6.1,62.3,145.800,1.26,79.6,111278,68350,25.930,12.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,309.045,2023,8,67618.666667,208262400.0,8.0,306.138,3.7,60.4,333.794,5.33,83.3,131434,80610,53.573,17.4
2023-09-01,311.176,2023,9,67738.333333,208262400.0,7.7,307.374,3.8,60.4,332.098,5.33,83.3,131434,80610,53.573,17.4
2023-10-01,313.262,2023,10,67858.000000,208262400.0,7.8,307.653,3.9,60.2,328.743,5.33,83.3,131434,80610,53.573,17.4


In [30]:
df.dropna(inplace=True)

In [31]:
df.isnull().sum()

CS_Index_Value       0
Year                 0
Month                0
Per_Capita_GDP       0
working_age_pop      0
Houses               0
CPI                  0
Unemployment_Rate    0
EmpRate              0
Cons_Material        0
FEDFUNDS             0
urban_pop_us         0
Num_Households       0
median_income        0
Subsidy              0
old_age_pop          0
dtype: int64

In [32]:
df

,CS_Index_Value,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Unemployment_Rate,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2003-01-01,128.461,2003,1,50462.000000,185656300.0,4.0,182.600,5.8,62.5,144.400,1.24,79.6,111278,68350,25.930,12.1
2003-02-01,129.356,2003,2,50573.333333,185830200.0,4.5,183.600,5.9,62.5,145.200,1.26,79.6,111278,68350,25.930,12.1
2003-03-01,130.148,2003,3,50684.666667,186120800.0,4.1,183.900,5.9,62.4,145.200,1.25,79.6,111278,68350,25.930,12.1
2003-04-01,130.884,2003,4,50796.000000,186475100.0,4.1,183.200,6.0,62.4,145.900,1.26,79.6,111278,68350,25.930,12.1
2003-05-01,131.735,2003,5,51034.666667,186696400.0,3.9,182.900,6.1,62.3,145.800,1.26,79.6,111278,68350,25.930,12.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,309.045,2023,8,67618.666667,208262400.0,8.0,306.138,3.7,60.4,333.794,5.33,83.3,131434,80610,53.573,17.4
2023-09-01,311.176,2023,9,67738.333333,208262400.0,7.7,307.374,3.8,60.4,332.098,5.33,83.3,131434,80610,53.573,17.4
2023-10-01,313.262,2023,10,67858.000000,208262400.0,7.8,307.653,3.9,60.2,328.743,5.33,83.3,131434,80610,53.573,17.4


In [33]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (252, 16)


This is our preprocessed datset. 

In [34]:
df.to_csv("prepared_dataset.csv")

In [35]:
us_house_price_df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
us_house_price_df.head()

,CS_Index_Value,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,Unemployment_Rate,EmpRate,Cons_Material,FEDFUNDS,urban_pop_us,Num_Households,median_income,Subsidy,old_age_pop
DATE,,,,,,,,,,,,,,,,
2003-01-01,128.461,2003,1,50462.000000,185656300.0,4.0,182.6,5.8,62.5,144.4,1.24,79.6,111278,68350,25.93,12.1
2003-02-01,129.356,2003,2,50573.333333,185830200.0,4.5,183.6,5.9,62.5,145.2,1.26,79.6,111278,68350,25.93,12.1
2003-03-01,130.148,2003,3,50684.666667,186120800.0,4.1,183.9,5.9,62.4,145.2,1.25,79.6,111278,68350,25.93,12.1
2003-04-01,130.884,2003,4,50796.000000,186475100.0,4.1,183.2,6.0,62.4,145.9,1.26,79.6,111278,68350,25.93,12.1
2003-05-01,131.735,2003,5,51034.666667,186696400.0,3.9,182.9,6.1,62.3,145.8,1.26,79.6,111278,68350,25.93,12.1


# To Be Continued...